In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/TFM\ BigData/ACCIDENTES/3_BUSQUEDA_Y_DATOS/data/data_prueba

/content/drive/MyDrive/TFM BigData/ACCIDENTES/3_BUSQUEDA_Y_DATOS/data/data_prueba


In [ ]:
!pip install dask[dataframe]
!pip install holidays

     |████████████████████████████████| 112kB 6.2MB/s 


In [ ]:
import csv
import pandas as pd
from datetime import datetime, time
import dask.dataframe as dd
import holidays
from dask import delayed
from dask.distributed import Client

In [ ]:
dask = dd.read_csv('2010-2018/*_Accidentalidad.csv', sep = ';', encoding = 'latin1')
dask

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS *,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
npartitions=9,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# Años del 2010 al 2018
years = range(2010, 2019)

# Creación DataFrame vacío
pandas = pd.DataFrame()

# Concatenación de los DataFrames
for year in years:
  print(year, '----------------------------')
  
  # Lectura de un csv
  df_aux = pd.read_csv(str(year) + '_Accidentalidad.csv', sep = ';', encoding = 'latin1')

  # Modificaciones en las variables
  if year == 2016:
    df_aux = df_aux.rename(columns = {'Nº VICTIMAS * ': 'Nº VICTIMAS *'})
  if year == 2018:
    df_aux = df_aux.rename(columns = {'* Nº VICTIMAS': 'Nº VICTIMAS *'})
  print('DF auxiliar:', df_aux.shape)

  # Se concatena el DataFrame auxiliar al DataFrame definitivo
  pandas = pd.concat([pandas, df_aux])
  print('DF:', pandas.shape)

pandas.head()

2010 ----------------------------
DF auxiliar: (26578, 26)
DF: (26578, 26)
2011 ----------------------------
DF auxiliar: (27342, 26)
DF: (53920, 26)
2012 ----------------------------
DF auxiliar: (26982, 26)
DF: (80902, 26)
2013 ----------------------------
DF auxiliar: (26839, 26)
DF: (107741, 26)
2014 ----------------------------
DF auxiliar: (27967, 26)
DF: (135708, 26)
2015 ----------------------------
DF auxiliar: (28172, 26)
DF: (163880, 26)
2016 ----------------------------
DF auxiliar: (29201, 26)
DF: (193081, 26)
2017 ----------------------------
DF auxiliar: (29795, 26)
DF: (222876, 26)
2018 ----------------------------
DF auxiliar: (30122, 26)
DF: (252998, 26)


,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,CPFA Niebla,CPFA Seco,CPFA Nieve,CPSV Mojada,CPSV Aceite,CPSV Barro,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,Nº VICTIMAS *,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,01/01/2010,DE 00:00 A 00:59,VIERNES,CHAMARTIN,CALLE DE CARTAGENA NUM ...,104,2010/135,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 18 A 20 AÑOS
1,01/01/2010,DE 00:00 A 00:59,VIERNES,CHAMARTIN,CALLE DE CARTAGENA NUM ...,104,2010/135,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1,COLISIÓN DOBLE,TURISMO,CONDUCTOR,HOMBRE,IL,DE 50 A 54 AÑOS
2,01/01/2010,DE 00:00 A 00:59,VIERNES,CHAMARTIN,CALLE DE CARTAGENA NUM ...,104,2010/135,NO,NO,SI,NO,NO,NO,SI,NO,NO,NO,NO,NO,1,COLISIÓN DOBLE,TURISMO,VIAJERO,MUJER,HL,DE 40 A 44 AÑOS
3,01/01/2010,DE 1:00 A 1:59,VIERNES,PUENTE DE VALLECAS,AUTOVIA M-30 CALZADA 2 KM. ...,10200,2010/94,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO,7,COLISIÓN MÚLTIPLE,NO ASIGNADO,VIAJERO,MUJER,HL,DE 25 A 29 AÑOS
4,01/01/2010,DE 1:00 A 1:59,VIERNES,PUENTE DE VALLECAS,AUTOVIA M-30 CALZADA 2 KM. ...,10200,2010/94,NO,NO,NO,NO,SI,NO,SI,NO,NO,NO,NO,NO,7,COLISIÓN MÚLTIPLE,NO ASIGNADO,VIAJERO,MUJER,HL,DE 30 A 34 ANOS


# Eficiencias

## Versión 1
Uso de Dask dataframe con la función map para la creación de las nuevas variables.

In [ ]:
%%time

dask['FECHA'] = dd.to_datetime(dask['FECHA'], format = '%d/%m/%Y')
dask['DIA'] = dask['FECHA'].map(lambda fecha: fecha.strftime('%d'))
dask['MES'] = dask['FECHA'].map(lambda fecha: fecha.strftime('%m'))
dask['ANO'] = dask['FECHA'].map(lambda fecha: fecha.strftime('%Y'))
dask['DIA SEMANA'] = dask['FECHA'].map(lambda fecha: fecha.weekday())
madrid_festivos = holidays.CountryHoliday('ES', prov='MD')
dask['FESTIVO'] = dask['FECHA'].map(lambda fecha: fecha in madrid_festivos)
dask[['FECHA', 'DIA', 'MES', 'ANO', 'DIA SEMANA', 'FESTIVO']].compute()

CPU times: user 10.6 s, sys: 310 ms, total: 10.9 s
Wall time: 12.7 s


## Versión 2 
Uso de Pandas dataframe con la función map para la creación de las nuevas variables.

In [ ]:
%%time

pandas['FECHA'] = dd.to_datetime(pandas['FECHA'], format = '%d/%m/%Y')
pandas['DIA'] = pandas['FECHA'].map(lambda fecha: fecha.strftime('%d'))
pandas['MES'] = pandas['FECHA'].map(lambda fecha: fecha.strftime('%m'))
pandas['ANO'] = pandas['FECHA'].map(lambda fecha: fecha.strftime('%Y'))
pandas['DIA SEMANA'] = pandas['FECHA'].map(lambda fecha: fecha.weekday())
madrid_festivos = holidays.CountryHoliday('ES', prov='MD')
pandas['FESTIVO'] = pandas['FECHA'].map(lambda fecha: fecha in madrid_festivos)
pandas[['FECHA', 'DIA', 'MES', 'ANO', 'DIA SEMANA', 'FESTIVO']]

CPU times: user 7.62 s, sys: 154 ms, total: 7.77 s
Wall time: 7.81 s


## Versión 3
Uso de Dask dataframe con la función map_partitions y una función de mapeo por cada nueva variable que se crea.

In [ ]:
%%time

dask['FECHA'] = dd.to_datetime(dask['FECHA'], format = '%d/%m/%Y')
dask['DIA'] = dask.map_partitions(lambda df_p: df_p['FECHA']
                                  .apply(lambda fecha: fecha.strftime('%d')))
dask['MES'] = dask.map_partitions(lambda df_p: df_p['FECHA']
                                  .apply(lambda fecha: fecha.strftime('%m')))
dask['ANO'] = dask.map_partitions(lambda df_p: df_p['FECHA']
                                  .apply(lambda fecha: fecha.strftime('%Y')))
dask['DIA SEMANA'] = dask.map_partitions(lambda df_p: df_p['FECHA']
                                         .apply(lambda fecha: fecha.weekday()))
madrid_festivos = holidays.CountryHoliday('ES', prov='MD')
dask['FESTIVO'] = dask.map_partitions(lambda df_p: df_p['FECHA']
                                .apply(lambda fecha: fecha in madrid_festivos))
dask[['FECHA', 'DIA', 'MES', 'ANO', 'DIA SEMANA', 'FESTIVO']].compute()

CPU times: user 10.7 s, sys: 244 ms, total: 10.9 s
Wall time: 10.2 s


## Versión 4 
Uso de Dask dataframe con la función map_partitions y una única función que mapeará con la función map por cada partición la creación de las nuevas variables.

In [ ]:
madrid_festivos = holidays.CountryHoliday('ES', prov='MD')

def formatear_fecha(df):
  df['DIA'] = df['FECHA'].map(lambda fecha: fecha.strftime('%d'))
  df['MES'] = df['FECHA'].map(lambda fecha: fecha.strftime('%m'))
  df['ANO'] = df['FECHA'].map(lambda fecha: fecha.strftime('%Y'))
  df['DIA SEMANA'] = df['FECHA'].map(lambda fecha: fecha.weekday())
  df['FESTIVO'] = df['FECHA'].map(lambda fecha: fecha in madrid_festivos)

  return df

In [ ]:
%%time

dask['FECHA'] = dd.to_datetime(dask['FECHA'], format = '%d/%m/%Y')
dask = dask.map_partitions(lambda df_p: formatear_fecha(df_p))
dask[['FECHA', 'DIA', 'MES', 'ANO', 'DIA SEMANA', 'FESTIVO']].compute()

CPU times: user 10.3 s, sys: 269 ms, total: 10.6 s
Wall time: 9.89 s


## Versión 5
Uso de Dask dataframe con la función map_partitions y una única función que mapeará con la función apply por cada partición la creación de las nuevas variables.

In [ ]:
madrid_festivos = holidays.CountryHoliday('ES', prov='MD')

def formatear_fecha(df):
  df['DIA'] = df['FECHA'].apply(lambda fecha: fecha.strftime('%d'))
  df['MES'] = df['FECHA'].apply(lambda fecha: fecha.strftime('%m'))
  df['ANO'] = df['FECHA'].apply(lambda fecha: fecha.strftime('%Y'))
  df['DIA SEMANA'] = df['FECHA'].apply(lambda fecha: fecha.weekday())
  df['FESTIVO'] = df['FECHA'].apply(lambda fecha: fecha in madrid_festivos)

  return df

In [ ]:
%%time

dask['FECHA'] = dd.to_datetime(dask['FECHA'], format = '%d/%m/%Y')
dask = dask.map_partitions(lambda df_p: formatear_fecha(df_p))
dask[['FECHA', 'DIA', 'MES', 'ANO', 'DIA SEMANA', 'FESTIVO']].compute()

CPU times: user 10.4 s, sys: 232 ms, total: 10.7 s
Wall time: 9.95 s
